In [3]:
#from google.colab import drive
#drive.mount('drive')

In [4]:


from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

In [5]:
def save_file(path,obj):
    pickle_out = open(path,"wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

In [6]:
#we don't need all descriptor to train Kmean or PCA,we can  just use some discriptors like 30 or 50 for each image
k_descriptors=30

#dataset's path 
dir_path_DataSet=''

#fold name for saving files(kmean/PCA)
fold_name="files/"
save_flod=''

docs=['miniMIT_Etus','chest_xray','kvasir-dataset-v2']#'kvasir-dataset-v2',]#,'chest_xray','kvasir-dataset-v2']#,'miniMIT_Etus','kvasir-dataset-v2']
scales=[0,1,2]
layers=['block5_pool']

In [7]:
for doc in docs:
    for layer in layers:
        for n in scales:
            print(doc, n , layer)
            #laod_data, we dont need to load all data 
            X=np.load(dir_path_DataSet + save_flod + doc +'X_train_'+layer + '(' + str(n) + ').npy')
            
            #this just for miniMIT bc we don't have many descriptorts 
            if doc=='miniMIT_Etus':
                k_descriptors=X.shape[1]*X.shape[2]
                


            #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
            d=X.shape[1]*X.shape[2]
            X = np.reshape(X,(X.shape[0],d,X.shape[3]))
            
            #we take only some discriptors
            if doc!='miniMIT_Etus':
                #take random descriptors
                indexes = np.arange(d)
                np.random.shuffle(indexes)
                indexes=indexes[:k_descriptors]
                X=X[:,indexes,:]
                
            
        
            if(n==0):
                x_all=X
            else:
                x_all=np.concatenate((x_all,X),axis=1)
                del X
            #x_test_all=np.concatenate((x_test_all,x_test),axis=1)

            print(x_all.shape)
      
      
      
        
        x_all=np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
        print(x_all.shape)
        #normalization l2
        x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)

        #PCA 
        pca = PCA(n_components=128)
        x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
        pca.fit(x_all)


        #SAVE the PCA  pickle
        save_file(ter_path+fold_name+doc+'_'+layer+'_'+'PCA.pickle',pca)

        #transformation with PCA
        print("PCA fit ...")
        x_train=pca.transform(x_all)
        del x_all

        #normalisation
        x_train = sklearn.preprocessing.normalize(x_train, norm = 'l2', axis = 1)
        print("pca shape")
        print(x_train.shape)

        #kmeans 64 clusters
        print("Kmean1 64 ...")
        kmeans1 = KMeans(n_clusters=64, random_state=0).fit(x_train)

        #SAVE the kmeans1  pickle
        save_file(ter_path+fold_name+doc+'_'+layer+'_'+'kmean1.pickle',kmeans1)

        print("Kmean2 256 ...")
        #kmeans 256 clusters
        kmeans2 = KMeans(n_clusters=256, random_state=0).fit(x_train)

        #SAVE the kmeans2  pickle
        save_file(ter_path+fold_name+doc+'_'+layer+'_'+'kmean2.pickle',kmeans2)
        print('end of ',doc , n , layer)
        
print("END")
      

kvasir-dataset-v2 0 block5_pool
float16
(6400, 7, 7, 512)
(6400, 30, 512)
(6400, 30, 512)
kvasir-dataset-v2 1 block5_pool
float16
(6400, 17, 17, 512)
(6400, 30, 512)
(6400, 60, 512)
kvasir-dataset-v2 2 block5_pool
float16
(6400, 27, 27, 512)
(6400, 30, 512)
(6400, 90, 512)
(576000, 512)


/home/farhat/.local/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/farhat/.local/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


PCA fit ...


/home/farhat/.local/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


pca shape
(576000, 128)
Kmean1 64 ...
Kmean2 256 ...
end of  kvasir-dataset-v2 2 block5_pool
END
